In [1]:
# Generate data for worst-case risk analysis.
srand(2);
n = 5;
r = abs(randn(n, 1))/15;
Sigma = 0.9 * rand(n, n) - 0.15;
Sigma_nom = Sigma' * Sigma;
Sigma_nom -= (maximum(Sigma_nom) - 0.9)

5x5 Array{Float64,2}:
 0.54178     0.45192   0.328892    0.0123621   0.400576
 0.45192     0.694173  0.416407   -0.237798    0.295939
 0.328892    0.416407  0.496781    0.0266768   0.31054 
 0.0123621  -0.237798  0.0266768   0.414028   -0.181312
 0.400576    0.295939  0.31054    -0.181312    0.9     

In [2]:
# Form and solve portfolio optimization problem.
# Here we minimize risk while requiring a 0.1 return.
using Convex, SCS
set_default_solver(SCSSolver(verbose=0));
w = Variable(n);
ret = dot(r, w);
risk = sum(quad_form(w, Sigma_nom));
problem = minimize(risk, [sum(w) == 1, ret >= 0.1, norm(w, 1) <= 2])
solve!(problem);
wval = vec(evaluate(w))

5-element Array{Float64,1}:
 -0.258385
  0.53885 
 -0.241623
  0.751627
  0.209527

In [3]:
# Form and solve worst-case risk analysis problem.
Sigma = Semidefinite(n);
Delta = Variable(n, n);
risk = sum(quad_form(wval, Sigma));
problem = maximize(risk, [Sigma == Sigma_nom + Delta,
                    diag(Delta) == 0,
                    abs(Delta) <= 0.2,
                    Delta == Delta']);
solve!(problem);
println("standard deviation = ", round(sqrt(wval' * Sigma_nom * wval), 2));
println("worst-case standard deviation = ", round(sqrt(evaluate(risk)), 2));
println("worst-case Delta = ");
println(round(evaluate(Delta), 2));

standard deviation = [0.27]
worst-case standard deviation = 0.82
worst-case Delta = 
[-0.0 -0.2 0.19 -0.2 -0.2
 -0.2 0.0 -0.2 0.2 0.2
 0.19 -0.2 -0.0 -0.2 -0.2
 -0.2 0.2 -0.2 0.0 0.2
 -0.2 0.2 -0.2 0.2 0.0]
